In [1]:
import numpy as np
from numpy import *
from random import shuffle
import math
import pandas as pd
from sklearn.metrics import r2_score

In [2]:
def data_generate(M,P_PDG,W,N,K,Noise):
    #W是原数据集,代表哪些代理之间有连接，即网络
    #N是W的行数，即代理的个数
    #代理人的博弈策略
    player = zeros((M+1,N))
    #原始博弈策略放在第一行，随机设置
    for i in range(N):
        if random.random()<=0.5:
            player[0,i] = 1 #1代表合作
        else:
            player[0,i] = 0 #0代表不合作
    #计算每个节点的收益
    #M轮博弈
    F = zeros((1,N))
    G = zeros((M,N))
    A = zeros((N,M,N))
    for t in range(M):
        for i in range(N):
            if player[t,i] == 1:
                s1 = array([[1],[0]])
            else:
                s1 = array([[0],[1]])
            for j in range(N):
                if player[t,j] == 0:
                    s2 = array([[0],[1]])
                else:
                    s2 = array([[1],[0]])
                F[0,j] = ((s1.T).dot(P_PDG)).dot(s2) #如果代理i和j连接，则代理i的收益为F[0,j]
            A[i,t,:] = F  #A:N*M*N   
            # F是三维矩阵A的第i页，第t行
            G[t,i] = F.dot(W[:,i]) #第t轮代理i的收益
        # update strategies
        for k in range(N):
            s=[i for i,x in enumerate(list(W[:,k])) if x>=1]     # 找出与代理k合作的代理的索引
            if len(s)!=0: # 如果有代理与代理k合作
                shuffle(s)
                P = 1/(1+math.e**((G[t,k]-G[t,s[0]])/K)) # 费米规则
                if random.random()<= P:
                    player[t+1,k] = player[t,s[0]]
                else:
                    player[t+1,k] = player[t,k]
            else: # 如果没有代理与代理k合作
                player[t+1,k] = player[t,k]
    # add noise for G
    Aa = G + Noise*random.random((M,N)) # m轮收益总矩阵：包含每一轮的收益M*N
    return [A,Aa]

In [3]:
def TrainSTRidge(R, Ut, lam, d_tol, maxit = 25, STR_iters = 10, l0_penalty = None, normalize = 2, split = 0.8, print_best_tol = False):
    """
    This function trains a predictor using STRidge.

    It runs over different values of tolerance and trains predictors on a training set, then evaluates them 
    using a loss function on a holdout set.

    Please note published article has typo.  Loss function used here for model selection evaluates fidelity using 2-norm,
    not squared 2-norm.
    """

    # Split data into 80% training and 20% test, then search for the best tolderance.
    np.random.seed(0) # for consistancy
    n,_ = R.shape
    train = np.random.choice(n, int(n*split), replace = False)
    test = [i for i in np.arange(n) if i not in train]
    TrainR = R[train,:]
    TestR = R[test,:]
    TrainY = Ut[train,:]
    TestY = Ut[test,:]
    D = TrainR.shape[1]       

    # Set up the initial tolerance and l0 penalty
    d_tol = float(d_tol)
    tol = d_tol
    if l0_penalty == None: l0_penalty = 0.001*np.linalg.cond(R)

    # Get the standard least squares estimator
    w = np.zeros((D,1))
    w_best = np.linalg.lstsq(TrainR, TrainY)[0]
    err_best = np.linalg.norm(TestY - TestR.dot(w_best), 2) + l0_penalty*np.count_nonzero(w_best)
    tol_best = 0

    # Now increase tolerance until test performance decreases
    for iter in range(maxit):

        # Get a set of coefficients and error
        w = STRidge(R,Ut,lam,STR_iters,tol,normalize = normalize)
        err = np.linalg.norm(TestY - TestR.dot(w), 2) + l0_penalty*np.count_nonzero(w)

        # Has the accuracy improved?
        if err <= err_best:
            err_best = err
            w_best = w
            tol_best = tol
            tol = tol + d_tol

        else:
            tol = max([0,tol - 2*d_tol])
            d_tol  = 2*d_tol / (maxit - iter)
            tol = tol + d_tol

    if print_best_tol: 
        print ("Optimal tolerance:", tol_best)
    return w_best

def STRidge(X0, y, lam, maxit, tol, normalize = 2, print_results = False):
    """
    Sequential Threshold Ridge Regression algorithm for finding (hopefully) sparse 
    approximation to X^{-1}y.  The idea is that this may do better with correlated observables.

    This assumes y is only one column
    """

    n,d = X0.shape
    X = np.zeros((n,d), dtype=np.complex64)
    # First normalize data
    if normalize != 0:
        Mreg = np.zeros((d,1))
        for i in range(0,d):
            Mreg[i] = 1.0/(np.linalg.norm(X0[:,i],normalize))
            X[:,i] = Mreg[i]*X0[:,i]
    else: X = X0
    
    # Get the standard ridge esitmate
    if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]
    else: w = np.linalg.lstsq(X,y)[0]
    num_relevant = d
    biginds = np.where( abs(w) > tol)[0]
    
    # Threshold and continue
    for j in range(maxit):

        # Figure out which items to cut out
        smallinds = np.where( abs(w) < tol)[0]
        new_biginds = [i for i in range(d) if i not in smallinds]
            
        # If nothing changes then stop
        if num_relevant == len(new_biginds): break
        else: num_relevant = len(new_biginds)
            
        # Also make sure we didn't just lose all the coefficients
        if len(new_biginds) == 0:
            if j == 0: 
                #if print_results: print "Tolerance too high - all coefficients set below tolerance"
                return w
            else: break
        biginds = new_biginds
        
        # Otherwise get a new guess
        w[smallinds] = 0
        if lam != 0: w[biginds] = np.linalg.lstsq(X[:, biginds].T.dot(X[:, biginds]) + lam*np.eye(len(biginds)),X[:, biginds].T.dot(y))[0]
        else: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]

    # Now that we have the sparsity pattern, use standard least squares to get w
    if biginds != []: w[biginds] = np.linalg.lstsq(X[:, biginds],y)[0]
    
    if normalize != 0: return np.multiply(Mreg,w)
    else: return w

In [4]:
M = 10 #m轮更新
b = 1.2 #叛逃者的收益
K = 0.1
P_PDG = array([[1,0],[b,0]]) #2*2的收益矩阵
Noise = 1
W = np.loadtxt("BA.txt") 
T = W.shape[0] #全部社区内个体的总数量
SV = 6
# the part for generate EG data
y = zeros((SV*M,T))
AA = zeros((T,SV*M,T))
for i in range(SV): #产生6组A和Y的值
    [A, Aa] = data_generate(M,P_PDG,W,T,K,Noise)
    for j in range(M):
        y[i*M+j,:] =Aa[j,:]
        for k in range(T):
            AA[k,i*M+j,:] = A[k,j,:]

In [5]:
print(Noise*random.random((M,T)))

[[0.77974539 0.92265754 0.94394586 0.6067525  0.70474156 0.53739137
  0.71518512 0.42823863 0.45367186 0.21289324 0.66176685 0.85772319
  0.48162597 0.52496948 0.83466943 0.0944519  0.44926796 0.70603554
  0.97086364 0.2443235  0.36945247 0.64409078 0.2718781  0.63300823
  0.0534163  0.29113102 0.43819243 0.28519265 0.91824105 0.45231649
  0.8320849  0.55107912 0.66402578 0.98269385 0.11150934 0.47618344
  0.72580086 0.96372572 0.02878708 0.09592467]
 [0.41794062 0.8655849  0.87187665 0.49373062 0.60814746 0.29447486
  0.2704046  0.68767202 0.87088826 0.88907676 0.6748374  0.29810121
  0.36947598 0.90392489 0.36083387 0.23725654 0.20673519 0.81815665
  0.72388027 0.87923426 0.8104604  0.70104446 0.9027695  0.49166855
  0.35629451 0.09041632 0.8437208  0.79382924 0.40870944 0.0766012
  0.44142603 0.76826877 0.65184869 0.43476488 0.88463317 0.84717249
  0.25907742 0.90367404 0.88516023 0.56562041]
 [0.89205674 0.68096633 0.71495388 0.83823614 0.36863169 0.05682301
  0.02736972 0.19950399

In [10]:
X=zeros((T,T))
for j in range(T):
    w = TrainSTRidge(AA[j], y[:,j].reshape(-1,1),10**-1,1)
    for i in range(w.shape[0]):
        X[i,j]=w[i,0]
np.savetxt("X.txt", X,fmt='%f',delimiter=' ')

<ipython-input-3-13ac3e2d7f51>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
<ipython-input-3-13ac3e2d7f51>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X) + lam*np.eye(d),X.T.dot(y))[0]
<ipython-input-3-13ac3e2d7f51>:102: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advis

In [11]:
def TrainSTRidge_SPL(R, Ut, v, lam, d_tol, maxit = 25, STR_iters = 10, l0_penalty = None, normalize = 2, split = 0.8, print_best_tol = False):
    """
    This function trains a predictor using STRidge_SPL.

    It runs over different values of tolerance and trains predictors on a training set, then evaluates them 
    using a loss function on a holdout set.

    Please note published article has typo.  Loss function used here for model selection evaluates fidelity using 2-norm,
    not squared 2-norm.
    """

    # Split data into 80% training and 20% test, then search for the best tolderance.
    np.random.seed(0) # for consistancy
    n,_ = R.shape
    train = np.random.choice(n, int(n*split), replace = False)
    test = [i for i in np.arange(n) if i not in train]
    TrainR = R[train,:]
    TestR = R[test,:]
    TrainY = Ut[train,:]
    TestY = Ut[test,:]
    D = TrainR.shape[1]       

    # Set up the initial tolerance and l0 penalty
    d_tol = float(d_tol)
    tol = d_tol
    if l0_penalty == None: l0_penalty = 0.001*np.linalg.cond(R)

    # Get the standard least squares estimator
    w = np.zeros((D,1))
    w_best = np.linalg.lstsq(TrainR, TrainY)[0]
    err_best = np.linalg.norm(TestY - TestR.dot(w_best), 2) + l0_penalty*np.count_nonzero(w_best)
    tol_best = 0

    # Now increase tolerance until test performance decreases
    for iter in range(maxit):

        # Get a set of coefficients and error
        w = STRidge_SPL(R,Ut,v,lam,STR_iters,tol,normalize = normalize)
        err = np.linalg.norm(TestY - TestR.dot(w), 2) + l0_penalty*np.count_nonzero(w)

        # Has the accuracy improved?
        if err <= err_best:
            err_best = err
            w_best = w
            tol_best = tol
            tol = tol + d_tol

        else:
            tol = max([0,tol - 2*d_tol])
            d_tol  = 2*d_tol / (maxit - iter)
            tol = tol + d_tol

    if print_best_tol: 
        print ("Optimal tolerance:", tol_best)
    return w_best

def STRidge_SPL(X0, y, v, lam, maxit, tol, normalize = 2, print_results = False):
    """
    Sequential Threshold Ridge Regression algorithm for finding (hopefully) sparse 
    approximation to X^{-1}y.  The idea is that this may do better with correlated observables.

    This assumes y is only one column
    """

    n,d = X0.shape
    X = np.zeros((n,d), dtype=np.complex64)
    # First normalize data
    if normalize != 0:
        Mreg = np.zeros((d,1))
        for i in range(0,d):
            Mreg[i] = 1.0/(np.linalg.norm(X0[:,i],normalize))
            X[:,i] = Mreg[i]*X0[:,i]
    else: X = X0
    
    # Get the standard ridge esitmate
    if lam != 0: w = np.linalg.lstsq(X.T.dot(X*v) + lam*np.eye(d),X.T.dot(y*v))[0]
    else: w = np.linalg.lstsq(X*v,y*v)[0]
    num_relevant = d
    biginds = np.where( abs(w) > tol)[0]
    
    # Threshold and continue
    for j in range(maxit):

        # Figure out which items to cut out
        smallinds = np.where( abs(w) < tol)[0]
        new_biginds = [i for i in range(d) if i not in smallinds]
            
        # If nothing changes then stop
        if num_relevant == len(new_biginds): break
        else: num_relevant = len(new_biginds)
            
        # Also make sure we didn't just lose all the coefficients
        if len(new_biginds) == 0:
            if j == 0: 
                #if print_results: print "Tolerance too high - all coefficients set below tolerance"
                return w
            else: break
        biginds = new_biginds
        
        # Otherwise get a new guess
        w[smallinds] = 0
        if lam != 0: w[biginds] = np.linalg.lstsq(X[:, biginds].T.dot(X[:, biginds]*v) + lam*np.eye(len(biginds)),X[:, biginds].T.dot(y*v))[0]
        else: w[biginds] = np.linalg.lstsq(X[:, biginds]*v,y*v)[0]

    # Now that we have the sparsity pattern, use standard least squares to get w
    if biginds != []: w[biginds] = np.linalg.lstsq(X[:, biginds]*v,y*v)[0]
    
    if normalize != 0: return np.multiply(Mreg,w)
    else: return w

In [17]:
X=zeros((T,T))
v = ones((SV*M,1))
v1 = zeros((SV*M,1))
Los = zeros((SV*M,T))
losss = zeros((SV*M,1))
lambda_0=0.3
t=0
while (t<7):  
    if (v==v1).all():
        break
    else:
        for j in range(T):
            w = TrainSTRidge_SPL(AA[j], y[:,j].reshape(-1,1), v,10**-1,1)
            Lo =  abs((np.dot(AA[j], w))-y[:,j].reshape(-1,1))
            for i in range(SV*M):
                Los[i,j]=Lo[i]
            for i in range(w.shape[0]):
                X[i,j]=w[i,0]
        for i in range(SV*M):
            losss[i]=np.mean(Los[i,:])
        t = t+1
        for i in range(SV*M):
            if losss[i]<lambda_0:
                v[i]=1-losss[i]/lambda_0
            else:
                v[i]=0
        lambda_0 = lambda_0*1.25
        #lambda_0 = lambda_0 + np.mean(losss)      
np.savetxt("X1.txt", X,fmt='%f',delimiter=' ')

<ipython-input-11-5e04d6e95ebc>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
<ipython-input-11-5e04d6e95ebc>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X*v) + lam*np.eye(d),X.T.dot(y*v))[0]
<ipython-input-11-5e04d6e95ebc>:102: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning w

In [18]:
v

array([[0.81470133],
       [0.79227534],
       [0.74429706],
       [0.82816424],
       [0.75089588],
       [0.79754892],
       [0.76682666],
       [0.80081071],
       [0.81129472],
       [0.79252522],
       [0.70329199],
       [0.75485742],
       [0.66294861],
       [0.71391277],
       [0.65433749],
       [0.58071264],
       [0.5528817 ],
       [0.5676372 ],
       [0.50820683],
       [0.59569646],
       [0.69107951],
       [0.73021421],
       [0.76877059],
       [0.71333769],
       [0.77436759],
       [0.68138501],
       [0.67098951],
       [0.79144733],
       [0.73231555],
       [0.75730053],
       [0.71831395],
       [0.7088039 ],
       [0.77961059],
       [0.78172301],
       [0.81553186],
       [0.79064592],
       [0.76816452],
       [0.75013183],
       [0.71976378],
       [0.78877083],
       [0.71035051],
       [0.68769356],
       [0.66507716],
       [0.6879143 ],
       [0.75619291],
       [0.73682551],
       [0.80254725],
       [0.695

In [42]:
X=zeros((T,T))
v = ones((SV*M,T))
v1 = zeros((SV*M,T))
Los = zeros((SV*M,T))
lambda_0=0.2
t=0
while (t<17):
    if (v==v1).all():
        break
    else:
        for j in range(T):
            w = TrainSTRidge_SPL(AA[j], y[:,j].reshape(-1,1), v[:,j].reshape(-1,1),10**-1,1)
            Lo =  abs((np.dot(AA[j], w))-y[:,j].reshape(-1,1))
            for i in range(SV*M):
                Los[i,j]=Lo[i]
            for i in range(w.shape[0]):
                X[i,j]=w[i,0]
        t = t+1
        for j in range(T):
            for i in range(SV*M):
                if Los[i,j]<lambda_0:
                    v[i,j]=1-Los[i,j]/lambda_0
                else:
                    v[i,j]=0
        lambda_0 = lambda_0*1.25
        #lambda_0 = lambda_0 + np.mean(Los)
np.savetxt("X2.txt", X,fmt='%f',delimiter=' ')

<ipython-input-11-5e04d6e95ebc>:30: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  w_best = np.linalg.lstsq(TrainR, TrainY)[0]
<ipython-input-11-5e04d6e95ebc>:76: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if lam != 0: w = np.linalg.lstsq(X.T.dot(X*v) + lam*np.eye(d),X.T.dot(y*v))[0]
<ipython-input-11-5e04d6e95ebc>:102: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning w

In [39]:
Los

array([[0.25547485, 0.07289031, 0.05733466, ..., 0.16423893, 0.19991899,
        0.09823141],
       [0.3920714 , 0.21839513, 0.27891199, ..., 0.14348697, 0.15999023,
        0.43737854],
       [0.17673184, 0.40307345, 0.00531019, ..., 0.11851106, 0.22347154,
        0.35153103],
       ...,
       [0.89991921, 0.21659801, 0.15316957, ..., 0.91817502, 0.13962206,
        0.91172765],
       [0.55609461, 0.30180643, 0.14699946, ..., 0.040398  , 0.18803909,
        0.97439674],
       [0.17481245, 0.21249521, 0.27805281, ..., 0.2917245 , 0.70662808,
        0.18617935]])